In [5]:
from PIL import Image, ImageDraw
import json
from PIL import Image
import torchvision.transforms as transforms
import torchvision

def draw_imgs(json_file_path, image_file_path):
   
    # JSON 파일 로드
    with open(json_file_path, 'r') as f:
        data = json.load(f)
        
    # bbox 데이터 가져오기
    annotations = data['annotations']
    height = float(data['images'][0]['height'])
    width = float(data['images'][0]['width'])
    # height = float(1960)
    # width = float(4032)
    print(height)
    print(width)

        
    # 이미지 열기
    image = Image.open(image_file_path)

    # Drawing context 생성
    draw = ImageDraw.Draw(image)

    # Bounding Box 그리기
    for annotaion in annotations:
        bbox = annotaion['bbox']
        x_min, y_min, w, h = bbox
        x_max = x_min + w 
        y_max = y_min + h

       
    draw.rectangle([(bbox[1], height - (bbox[0] + bbox[2]),bbox[1] + bbox[3], height- bbox[0])], outline="red", width=3)
    # 결과 저장
    output_path = "./test/draw/result.jpg"
    image.save(output_path)

    print(f"이미지 저장 완료: {output_path}")



In [6]:
def draw_images_for_file(f_name):
    # JSON 파일 경로 설정
    json_file_path = './datasets/test1/' + f_name + '.json'
    print(json_file_path)
    # 이미지 파일 경로 설정
    image_file_path = './datasets/test1/' + f_name + '.jpg'
    print(image_file_path)
    
    # draw_imgs 함수 호출
    draw_imgs(json_file_path, image_file_path)

# 파일 이름 설정
f_name = '07_20201028_567302'

# 함수 호출
draw_images_for_file(f_name)

./datasets/test1/07_20201028_567302.json
./datasets/test1/07_20201028_567302.jpg
1960.0
4032.0
이미지 저장 완료: ./test/draw/result.jpg


In [4]:
import os
import json
from PIL import Image
import torchvision.transforms as transforms
import torchvision

# 크롭된 이미지를 저장할 기본 경로
crop_base_path = './rose/rose_img_crop'

# JSON 파일에서 bbox 데이터를 사용하여 이미지를 크롭하는 함수
def crop_images(json_file_path, image_file_path):
    # JSON 파일 로드
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    
    # 이미지 파일 열기
    img = Image.open(image_file_path)
    
    # 변환할 때 사용할 크롭 및 변환 작업 정의
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    # bbox 데이터 가져오기
    annotations = data['annotations']
    categories = {category['id']: category['name'] for category in data['categories']}
    height = float(data['images'][0]['height'])
    width = float(data['images'][0]['width'])
    
    cropped_images = []
    # bbox를 사용하여 이미지 크롭 및 저장
    for annotation in annotations:
        bbox = annotation['bbox']
        
        # 이미지를 크롭합니다.
        cropped_img = img.crop((bbox[1], height - (bbox[0] + bbox[2]),bbox[1] + bbox[3], height- bbox[0]))
        cropped_img = transform(cropped_img)  # Torch 텐서로 변환
        cropped_images.append(cropped_img)
        
        # 어노테이션 ID에 해당하는 카테고리 이름 가져오기
        category_id = annotation['category_id']
        category_name = categories[category_id]
        
        # 카테고리 이름에 해당하는 폴더 생성
        category_path = os.path.join(crop_base_path, category_name)
        if not os.path.exists(category_path):
            os.makedirs(category_path)
        
        # 이미지 저장
        file_name = os.path.splitext(os.path.basename(image_file_path))[0] + '_' + str(annotation['id']) + '.jpg'
        save_path = os.path.join(category_path, file_name)
        torchvision.utils.save_image(cropped_img, save_path)
        print(f"이미지 저장 완료: {save_path}")

    return cropped_images



In [86]:
def crop_images_for_file(f_name):
    # JSON 파일 경로 설정
    json_file_path = './rose/rose_label/' + f_name + '.json'
    print(json_file_path)
    # 이미지 파일 경로 설정
    image_file_path = './rose/rose_img/' + f_name + '.jpg'
    print(image_file_path)
    
    # crop_images 함수 호출
    cropped_images = crop_images(json_file_path, image_file_path)

# 파일 이름 설정
f_name = '07_20201028_567425'

# 함수 호출
crop_images_for_file(f_name)

./rose/rose_label/07_20201028_567425.json
./rose/rose_img/07_20201028_567425.jpg
이미지 저장 완료: ./rose/rose_img_crop\줄기\07_20201028_567425_0.jpg
이미지 저장 완료: ./rose/rose_img_crop\화방\07_20201028_567425_1.jpg
